In [1]:
import os

from langchain.chains import RetrievalQA
from PyPDF2 import PdfReader
from langchain.callbacks.base import BaseCallbackHandler
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Neo4jVector
from langchain_community.chat_models import ChatOllama
from langchain_core.documents import Document
import os
import json
from semantic_text_splitter import TextSplitter
# load api key lib
from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain.chains import RetrievalQA,RetrievalQAWithSourcesChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain.storage import InMemoryStore
from langchain.vectorstores import FAISS
import os


embeddings_options=["medllama2","huggingface"]

embeddings_name=embeddings_options[0]

load_dotenv(".env")

url = "neo4j://localhost:7687"
# url = "neo4j://localhost:7474"
username ="neo4j"
password = "password"

os.environ["NEO4J_URI"] = url #"bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] =username # "neo4j"
os.environ["NEO4J_PASSWORD"] = password #"password"


url = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")
ollama_base_url = os.getenv("OLLAMA_BASE_URL")
embedding_model_name = os.getenv("EMBEDDING_MODEL")
llm_name = "medllama2"#os.getenv("LLM")
# Remapping for Langchain Neo4j integration
os.environ["NEO4J_URL"] = url



if(embeddings_name==embeddings_options[0]):
    embeddings = OllamaEmbeddings(
                model="medllama2"
            )
if(embeddings_name==embeddings_options[1]):
    # model = "sentence-transformers/all-mpnet-base-v2"
    model = "ncbi/MedCPT-Query-Encoder"
    embeddings = HuggingFaceHubEmbeddings(
    model=model,
    task="embedding",
    huggingfacehub_api_token="xxx",
)

max_characters = 200
# Optionally can also have the splitter not trim whitespace for you
splitter = TextSplitter()

# Document(page_content=sect['text'])

# folder_path = "/workspaces/thyroid_knowledge_graph/preprocessed"
folder_path = "/workspaces/thyroid_knowledge_graph/preprocessed_full_new_biblio"
file_paths = []

for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)

texts=[]
texts_dict={}
for file_path in file_paths:
    index=0
    with open(file_path, 'r') as file:
        data = json.load(file)
        try:
            sections= data['sections']
            data_loc=[]

            for sect in sections:
                
                # keyy=f"{os.path.basename(file_path)}___{index}"
                # texts.append(Document(page_content=sect['text'],metadata={"source":keyy ,"num":index}))   
                # texts_dict[keyy]=sect['text']
                # res.append(sect['text'])
                for chunk in splitter.chunks(sect['text'], max_characters):
                    index=index+1
                    keyy=f"{os.path.basename(file_path)}___{index}"
                    texts.append(Document(page_content=chunk,metadata={"source": keyy,"num":index})) 
                    texts_dict[keyy]=sect['text']

        except:
            print("error")
                
            # data_loc.append(sect['text'],)
        # data_loc="\n".join(data_loc)
        # res.append(Document(page_content=data_loc,metadata={"source": file_path}))     
        


llm_med = ChatOllama(model="medllama2")

# texts=texts[0:100] #TODO remove


#memory for retriever
# memory = ConversationSummaryBufferMemory(llm=llm_med, input_key='question', output_key='answer', return_messages=True)


error
error


In [2]:
texts

[Document(page_content='The putative first description of C cells can be attributed to E.', metadata={'source': 'Histopathology of C Cells and Medullary Thyroid Carcinoma.json___1', 'num': 1}),
 Document(page_content='Cresswell Baber in the thyroid gland of the dog in 1876; he suspected that sheets of cells situated between the "vesicles" (i.e. follicles) represent the regeneration pool of the "vesicles" (Baber', metadata={'source': 'Histopathology of C Cells and Medullary Thyroid Carcinoma.json___2', 'num': 2}),
 Document(page_content='1876). In 1894, Karl Hürthle described in his only publication dealing with the thyroid cell complexes in dog thyroid which he called "interfollicular epithelium" (Hürthle 1894).', metadata={'source': 'Histopathology of C Cells and Medullary Thyroid Carcinoma.json___3', 'num': 3}),
 Document(page_content='Later, these cells were confused with follicular-derived elements such as oncocytic cells, leading particularly in the Anglo-American usage to the sti

In [3]:
# # metas=list(map(lambda el: f"{el.metadata['source']}___{el.metadata['num']}",texts))
# metas=list(map(lambda el: el.metadata,texts))
# metas

In [4]:
# vectorstore = Chroma(embedding_function=embeddings, persist_directory="/data")
# vectorstore = FAISS.from_documents(list(map(lambda el: el.page_content,texts)), embedding=embeddings)
vectorstore = FAISS.from_documents(texts, embedding=embeddings)
vectorstore.save_local("/data/faiss_index")
# This text splitter is used to create the child documents
# child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# store = InMemoryStore()
# retriever = vectorstore.as_retriever(search_kwargs=dict(k=5))
retriever = vectorstore.as_retriever()

# # store = InMemoryStore()
# retriever = ParentDocumentRetriever(
#     vectorstore=vectorstore,
#     docstore=store,
#     child_splitter=child_splitter,
# )
# retriever.add_documents(res, ids=metas)

In [ ]:
# chunks = res[0:2]
# chunks=res

vector_qa = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm_med,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)


# https://github.com/langchain-ai/langchain/issues/5642

In [ ]:
result = vector_qa({"question": "as an expert in endocrinology and nuclear medicine what are indications to monitor thyroid cancer patients with thyroglobulin and what are the limitations of thyroglobulin as a tumor marker?"})
result

In [ ]:
# Store the chunks part in db (vector)

# vectorstore = Neo4jVector.from_texts(
#     chunks,
#     url=url,
#     username=username,
#     password=password,
#     embedding=embeddings,
#     index_name="pdf_bot",
#     node_label="PdfBotChunk",
#     pre_delete_collection=True,  # Delete existing PDF data
# )

# vectorstore = Chroma(embedding_function=embeddings, persist_directory="/data")



In [ ]:
# vector_qa = RetrievalQAWithSourcesChain.from_chain_type(
#     llm=llm_med,
#     chain_type="stuff",
#     retriever=retriever
# )


In [ ]:
# result = vector_qa({"question": "what is RIT"})
# result